# Setup

In [1]:
from dataclasses import dataclass


In [2]:
!pip install numpy-financial 

# Inputs

Some inputs were given in the problem definition. List those here.

In [31]:
# Do not remove or modify any existing input names, though new inputs can be added
@dataclass
class ModelInputs:
    bond_years: int = 15
    bond_coupon: float = .0525
    bond_price: float = 130.58
    bond_par: float = 100
    risk_free: float = 0.005
    price: float = 119.51
    shares_outstanding: int = 2850000000
    libor_rate: float = 0.0196
    sp_500_file: str = 'SP500 Prices.xlsx'
    stock_prices_file: str = 'WMT Prices.xlsx'
    stock_ticker: str = 'WMT'
    balance_sheet_file: str = 'WMT Balance Sheet.xlsx'
    income_statement_file: str = 'WMT Income Statement.xlsx'
    mv_debt_details_file: str = 'WMT Debt Details.xls'
        
        
@dataclass
class SimulationInputs:
    num_iter: int = 10000
    beta_std: float = 0.2
    mkt_ret_std: float = 0.03
    bond_price_std: float = 30
    tax_rate_std: float = 0.05

model_data = ModelInputs()
sim_data = SimulationInputs()
data = model_data
data 


ModelInputs(bond_years=15, bond_coupon=0.0525, bond_price=130.58, bond_par=100, risk_free=0.005, price=119.51, shares_outstanding=2850000000, libor_rate=0.0196, sp_500_file='SP500 Prices.xlsx', stock_prices_file='WMT Prices.xlsx', stock_ticker='WMT', balance_sheet_file='WMT Balance Sheet.xlsx', income_statement_file='WMT Income Statement.xlsx', mv_debt_details_file='WMT Debt Details.xls')

In [4]:
# Add your model logic, sections, and as many cells as you want here.

In [32]:
import pandas as pd 
df = pd.read_excel('Walmartdata (1).xlsx')
df.head()

,Market Returns,Asset Returns
0,2031.209961,68.586304
1,2031.920044,69.432503
2,2038.260010,70.023087
3,2039.680054,69.644073
4,2038.250000,69.811539


We will now convert the prices into returns 

In [6]:
returns = df.pct_change()
returns.head()

,Market Returns,Asset Returns
0,NaN,NaN
1,0.000350,0.012338
2,0.003120,0.008506
3,0.000697,-0.005413
4,-0.000701,0.002405


## Calculating the Market Risk Premium

We are ultimately going to be running a regression to determine $\beta$. We can think of a standard regression line as following the equation: $$y = a + bx$$ We can put the CAPM in this format if we assume $\epsilon$ is zero, then treat $r_i$ as $y$, $r_f$ as $a$, and $(r_m - r_f)$ as $x$. Therefore we need to calculate the market risk premium (MRP), $(r_m - r_f)$, to use in the regression. 

From the problem, the risk free rate is 3%. So just subtract that from the market returns to get the MRP.

In [7]:
risk_free = data.risk_free / 252
returns['MRP'] = returns['Market Returns'] - risk_free
returns.head()

,Market Returns,Asset Returns,MRP
0,NaN,NaN,NaN
1,0.000350,0.012338,0.000330
2,0.003120,0.008506,0.003100
3,0.000697,-0.005413,0.000677
4,-0.000701,0.002405,-0.000721


We will now calculate the Beta. 

In [8]:
returns = returns.dropna()
returns.head()

,Market Returns,Asset Returns,MRP
1,0.000350,0.012338,0.000330
2,0.003120,0.008506,0.003100
3,0.000697,-0.005413,0.000677
4,-0.000701,0.002405,-0.000721
5,0.000530,0.047222,0.000510


In [9]:
import statsmodels.api as sm
X = sm.add_constant(returns['MRP'])
y = returns['Asset Returns']

model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Asset Returns   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     280.6
Date:                Thu, 26 Sep 2024   Prob (F-statistic):           5.38e-57
Time:                        16:06:33   Log-Likelihood:                 3854.8
No. Observations:                1257   AIC:                            -7706.
Df Residuals:                    1255   BIC:                            -7695.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.000      0.938      0.348      -0.000       0.001
MRP            0.6248      0.037     16.751      0.000       0.552       0.698
==============================================================================
Omnibus:                      383.484   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26747.784
Skew:                           0.480   Prob(JB):                         0.00
Kurtosis:                      25.578   Cond. No.                         117.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
beta = results.params['MRP']
beta

0.624832446531551

## Estimating Market Return 

In [11]:
market_return = returns['Market Returns'].mean()
print(market_return)

0.0003662986652878659


In [12]:
capm = risk_free + beta * (market_return - risk_free)
cost_of_equity = capm * 252 #we multiply by 252 to get the yearly CAPM 
print(f'The cost of equity is {cost_of_equity:.2%}.')

The cost of equity is 5.96%.


## Estimating Market value of equity 

In [13]:
mv_of_equity = data.price * data.shares_outstanding 
print (f'The market value of equity is {mv_of_equity:,.2f}.')

The market value of equity is 340,603,500,000.00.


## Estimating Cost of debt 

In [14]:
import numpy_financial as npf
import pandas as pd 

In [15]:
coupon = data.bond_coupon
price = data.bond_price 
n = data.bond_years
face_value = data.bond_par



In [16]:
payment = face_value * coupon 
payment

5.25

In [17]:
pre_taxcod = npf.rate(n,payment,-price,face_value)
pre_taxcod

0.027369148593484927

In [18]:
bsdf = pd.read_excel('WMT Balance Sheet.xlsx', index_col=0)
isdf = pd.read_excel('WMT Income Statement.xlsx',index_col = 0)
date = isdf[pd.to_datetime('2019-07-31')]
date2= bsdf[pd.to_datetime('2019-07-31')]

In [19]:
tax_paid= date ['Income Tax Expense']
tax_paid

1233000000.0

In [2]:
ebt = date['Earnings before Tax']
ebt

NameError: name 'date' is not defined

In [21]:
tax_rate = tax_paid / ebt
tax_rate 

0.25459425975634936

In [22]:
aftertax_cod = pre_taxcod * (1-tax_rate)
aftertax_cod     

0.020401120467165103

In [23]:
print(f'The pre-tax cost of debt is {pre_taxcod:.2%}, the after-tax cost is {aftertax_cod:.2%}')

The pre-tax cost of debt is 2.74%, the after-tax cost is 2.04%


## Estimating Market value of debt

In [24]:
!pip install --upgrade openpyxl



In [25]:
mv_of_debt = 82700000000

## Weight Of Equity and debt

In [26]:
mv_of_debt

82700000000

In [27]:
sum_of_weights = mv_of_equity + mv_of_debt
weight_of_equity = mv_of_equity / sum_of_weights
weight_of_debt = mv_of_debt / sum_of_weights 

print(weight_of_debt,weight_of_equity)

0.19536809877546488 0.8046319012245351


In [28]:
equity=weight_of_equity*cost_of_equity
debt=aftertax_cod*weight_of_debt
weighted_av = equity + debt

weighted_av

0.05190349792307415

## Main answer for baseline case:
coe=5.96%
mv_equity = 340,603,500,000.00
pretax_cost_of_debt = 2.74%
aftertax_cost_of_debt = 2.04%
mv_debt = 82,700,000,000
wacc = 5.19%


# Monte carlo simulation on wacc

A Monte Carlo simulation is a way to model the probability of different outcomes in a process that cannot easily be predicted due to the intervention of random variables. 

In [33]:
import numpy as np
import pandas as pd

# Define the SimulationInputs object
sim_inputs = SimulationInputs()

# Use sim_inputs in the Monte Carlo simulation
iterations = sim_inputs.num_iter

# Example means (you can adjust as necessary)
beta_mean = 1.2
market_return_mean = 0.08
bond_price_mean = 100
tax_rate_mean = 0.3

# Arrays to store simulation results using the standard deviations from SimulationInputs
simulations = {
    'Beta': np.random.normal(beta_mean, sim_inputs.beta_std, iterations),
    'Market Return': np.random.normal(market_return_mean, sim_inputs.mkt_ret_std, iterations),
    'Bond Price': np.random.normal(bond_price_mean, sim_inputs.bond_price_std, iterations),
    'Tax Rate': np.random.normal(tax_rate_mean, sim_inputs.tax_rate_std, iterations)
}

# Convert to DataFrame
sim_df = pd.DataFrame(simulations)

# Calculate WACC and other financial metrics as before
sim_df['Cost of Equity'] = 0.05 + sim_df['Beta'] * (sim_df['Market Return'] - 0.03)  # Example CAPM formula
sim_df['Cost of Debt'] = 0.04 + 0.01 * (sim_df['Bond Price'] / 100)

# Example market value of equity and debt
mv_equity = 340603500000  
mv_debt = 82700000000
total_value = mv_equity + mv_debt
weight_of_equity = mv_equity / total_value
weight_of_debt = mv_debt / total_value

# Calculate WACC
sim_df['WACC'] = (weight_of_equity * sim_df['Cost of Equity']) + (weight_of_debt * sim_df['Cost of Debt'] * (1 - sim_df['Tax Rate']))

# Summary statistics for WACC
wacc_summary = sim_df['WACC'].describe()

# Display the results
wacc_summary


count    10000.000000
mean         0.095455
std          0.030658
min         -0.025286
25%          0.074640
50%          0.094001
75%          0.114502
max          0.248597
Name: WACC, dtype: float64

In [35]:
print(f'{len(sim_df)} simulations were run.')

10000 simulations were run.


In [42]:
styled_sim_df = sim_df.head().style\
    .background_gradient(cmap="coolwarm", subset=['WACC', 'Cost of Equity', 'Cost of Debt'])\
    .highlight_min(color='lightgreen', subset=['WACC', 'Cost of Equity', 'Cost of Debt'])\
    .highlight_max(color='lightcoral', subset=['WACC', 'Cost of Equity', 'Cost of Debt'])

# Display the styled DataFrame (works in a Jupyter notebook)
styled_sim_df

,Beta,Market Return,Bond Price,Tax Rate,Cost of Equity,Cost of Debt,WACC
0,0.947572,0.039940,150.050732,0.213073,0.059419,0.055005,0.056267
1,1.185301,0.105628,37.131114,0.344244,0.139642,0.043713,0.117960
2,1.118951,0.043761,139.542086,0.338229,0.065398,0.053954,0.059597
3,0.951545,0.053755,68.490924,0.285638,0.072604,0.046849,0.064958
4,1.203773,0.081564,74.820435,0.268352,0.112071,0.047482,0.096963
